In [2]:
%pip install keplergl

     ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
     --------------------------------------- 0.0/18.4 MB 330.3 kB/s eta 0:00:56
     --------------------------------------- 0.0/18.4 MB 393.8 kB/s eta 0:00:47
     --------------------------------------- 0.1/18.4 MB 655.4 kB/s eta 0:00:28
      --------------------------------------- 0.4/18.4 MB 2.4 MB/s eta 0:00:08
     - -------------------------------------- 0.8/18.4 MB 3.6 MB/s eta 0:00:05
     -- ------------------------------------- 1.2/18.4 MB 4.6 MB/s eta 0:00:04
     --- ------------------------------------ 1.8/18.4 MB 5.6 MB/s eta 0:00:03
     ---- ----------------------------------- 2.3/18.4 MB 6.4 MB/s eta 0:00:03
     ------ --------------------------------- 2.8/18.4 MB 7.0 MB/s eta 0:00:03
     ------- -------------------------------- 3.4/18.4 MB 7.4 MB/s eta 0:00:03
     -------- ------------------------------- 3.9/18.4 MB 8.1 MB/s eta 0:00:02
     --------- ------------------------------ 4.5/18.4 M


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install pydeck pandas h3 geopandas

Note: you may need to restart the kernel to use updated packages.Collecting pydeck
  Obtaining dependency information for pydeck from https://files.pythonhosted.org/packages/ab/4c/b888e6cf58bd9db9c93f40d1c6be8283ff49d88919231afe93a6bcf61626/pydeck-0.9.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for h3 from https://files.pythonhosted.org/packages/34/8f/7d5905e44f8aa5ca047d54737a37267e10be2108b455689539611b59b6ff/h3-4.1.2-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/6.9 MB 1.2 MB/s eta 0:00:06
   - -------------------------------------- 0.3/6.9 MB 2.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/6.9 MB 4.0 MB/s eta 0:00:02
   ------ --------------------------------- 1.1/6.9 MB 5.3 MB/s eta 0:00:02
   ------- -------------------------------- 1.3/6.9 MB 5.8 MB/s eta 0:00:01



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import geopandas as gpd
from keplergl import KeplerGl

In [11]:
class Disease:
    def __init__(self):
        self.transmission_rate = 0.01
        self.incubation_rate = 0.05
        self.recovery_rate = 0.01
        self.mortality_rate = 0.015

    def seir_rates(self):
        return self.transmission_rate, self.incubation_rate, self.recovery_rate, self.mortality_rate

In [13]:
import random


class PandemicSpreadSimulation:
    def __init__(self):
        gdf = gpd.read_file('kontur_population_20231101_r4.gpkg')

        gdf = gdf[gdf['population'] > 50]

        # Upewnij się, że dane są w odpowiednim układzie współrzędnych (WGS84)
        gdf = gdf.to_crs(epsg=4326)

        # Dodaj pola SEIR do danych
        gdf['S'] = gdf['population']  # Wszyscy początkowo podatni
        gdf['E'] = 0  # Początkowo brak osób narażonych
        gdf['I'] = 0  # Początkowo brak osób zakażonych
        gdf['R'] = 0  # Początkowo brak ozdrowieńców
        gdf['V'] = 0  # Początkowo brak zaszczepionych
        gdf['airport'] = False
        gdf['masks'] = False
        gdf['lockdown'] = 0
        gdf['vaccinations'] = False

        target_hex_id = '846026bffffffff'
        target_hex = gdf[gdf['h3'] == target_hex_id]

        if not target_hex.empty:
            # Przenieś połowę osób z S do I
            half_S = target_hex['S'].values[0] // 2
            gdf.loc[gdf['h3'] == target_hex_id, 'I'] = half_S
            gdf.loc[gdf['h3'] == target_hex_id, 'S'] = target_hex['S'].values[0] - half_S
        else:
            print('No target hex found')

        self.gdf = gdf
        self.disease = Disease()

    def infect_neighbors(self):
        for index, row in self.gdf.iterrows():
            infected = row['I']
            if infected == 0:
                continue

            neighbors = self.gdf[self.gdf.geometry.touches(row.geometry)]

            for neighbor_index, neighbor in neighbors.iterrows():
                neighbor_S = neighbor['S']
                neighbor_population = neighbor['population']

                transmission_probability = 1 - ((1 - infected / row['population']) ** 5)

                is_transmitted = random.random() < transmission_probability
                if is_transmitted:
                    # Przenieś część osób z S do E w sąsiedniej populacji
                    new_exposed = random.randint(1, 10)  # Zakładamy losową liczbę nowych narażonych
                    actual_exposed = min(new_exposed, neighbor_S)  # Nie więcej niż dostępni w S

                    self.gdf.at[neighbor_index, 'S'] -= actual_exposed
                    self.gdf.at[neighbor_index, 'E'] += actual_exposed

        print("Neighbor infections processed.")

    # update simulation state by a day
    def update_simulation(self):
        self.infect_neighbors()

        flights_per_day = 10000
        for _ in range(flights_per_day):
            self.simulate_flight()

        transmission_rate, incubation_rate, recovery_rate, mortality_rate = self.disease.seir_rates()

        for index, row in self.gdf.iterrows():
            S = row['S']
            E = row['E']
            I = row['I']
            R = row['R']
            population = row['population']
            lockdown_days = row['lockdown']

            if lockdown_days > 0:
                self.gdf.at[index, 'lockdown'] = lockdown_days - 1

            # lockdown condition
            if I > population * 0.2 and lockdown_days == 0:
                self.gdf.at[index, 'lockdown'] = 30

            lockdown_transmission_reduce_rate = 0.01
            transmission_rate *= lockdown_transmission_reduce_rate

            new_exposed = int(transmission_rate * S * I / population)
            new_infected = int(incubation_rate * E)
            new_recovered = int(recovery_rate * I)
            new_dead = int(mortality_rate * I)

            # Zaktualizuj wartości SEIR
            new_S = S - new_exposed
            if new_S < 0:
                new_S = 0

            new_E = E + new_exposed - new_infected
            if new_E < 0:
                new_E = 0

            new_I = I + new_infected - new_recovered - new_dead
            if new_I < 0:
                new_I = 0

            new_R = R + new_recovered
            if new_R < 0:
                new_R = 0

            # Uaktualnij dane w gdf
            self.gdf.at[index, 'S'] = new_S
            self.gdf.at[index, 'E'] = new_E
            self.gdf.at[index, 'I'] = new_I
            self.gdf.at[index, 'R'] = new_R

    def simulate_flight(self):
        # Wybierz populacje z lotniskami bez lockdownu
        airports = self.gdf[(self.gdf['airport'] == True) & (self.gdf['lockdown'] == False)]
        if len(airports) < 2:
            return

        origin, destination = airports.sample(2)

        passengers = random.randint(100, 300)

        # Oblicz prawdopodobieństwo, że samolot jest zainfekowany
        infected_probability = 1 - ((1 - origin['E'] / origin['population']) ** passengers)

        # Losuj, czy samolot jest zainfekowany
        is_infected = random.random() < infected_probability

        if is_infected:
            # Jeśli samolot jest zainfekowany, przenieś pasażerów z S do E w docelowej populacji
            destination_S = self.gdf.at[destination.name, 'S']
            new_exposed = min(passengers, destination_S)  # Nie więcej niż S

            self.gdf.at[destination.name, 'S'] -= new_exposed
            self.gdf.at[destination.name, 'E'] += new_exposed

        print(f"Flight from {origin['h3']} to {destination['h3']}")
        print(f"Passengers: {passengers}")
        print(f"Plane infected: {'Yes' if is_infected else 'No'}")

In [4]:
def create_map_with_seir(gdf, filename='kepler_map_seir.html'):
    # Wczytaj dane granic państw
    countries = gpd.read_file("ne_110m_admin_0_countries.shp")  # Granice państw z Natural Earth
    countries.set_crs(epsg=4326, inplace=True)

    # Wczytaj dane miast
    cities = gpd.read_file("ne_110m_populated_places.shp")  # Miasta z Natural Earth
    cities.set_crs(epsg=4326, inplace=True)

    # Zaktualizuj konfigurację mapy
    config = {
        "version": "v1",
        "config": {
            "visState": {
                "filters": [],
                "layers": [
                    # Warstwa H3 Hexagony (jeśli używasz danych H3)
                    {
                        "id": "h3layer",
                        "type": "hexagonId",
                        "config": {
                            "dataId": "population_data",
                            "label": "H3 Hexagons",
                            "color": [0, 255, 0],  # Początkowy kolor zielony
                            "columns": {
                                "hex_id": "h3_index"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "colorRange": {
                                    "name": "Custom Gradient",
                                    "type": "sequential",
                                    "category": "Custom",
                                    "colors": ["#00FF00", "#FFA500"]  # Od zielonego do pomarańczowego
                                },
                                "coverage": 1,
                                "enable3d": True,
                                "sizeRange": [0, 500],
                                "coverageRange": [0, 1],
                                "elevationScale": 5
                            },
                        },
                        "visualChannels": {
                            "colorField": {
                                "name": "I",  # Użyj liczby zakażonych jako podstawy koloru
                                "type": "real"
                            },
                            "colorScale": "quantile",
                            "sizeField": {
                                "name": "population",
                                "type": "real"
                            },
                            "sizeScale": "linear"
                        }
                    },
                    # Warstwa GeoJSON (jeśli chcesz wyświetlać granice państw w formacie GeoJSON)
                    {
                        "id": "geojsonlayer",
                        "type": "geojson",
                        "config": {
                            "dataId": "countries",
                            "label": "Countries",
                            "color": [200, 200, 200],  # Ustaw kolor konturów na szary
                            "columns": {
                                "geojson": "geometry"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "colorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": ["#D3D3D3", "#D3D3D3"]  # Jasno szary
                                },
                                "coverage": 1
                            }
                        }
                    },
                    # Warstwa Miast
                    {
                        "id": "cityLayer",
                        "type": "geojson",
                        "config": {
                            "dataId": "cities",
                            "label": "Cities",
                            "color": [255, 0, 0],  # Kolor czerwony dla miast
                            "columns": {
                                "geojson": "geometry"
                            },
                            "isVisible": True
                        }
                    }
                ]
            },
            "mapState": {
                "bearing": 0,
                "latitude": 0,
                "longitude": 0,
                "pitch": 45,
                "zoom": 2,
                "dragRotate": True
            },
            "mapStyle": {
                "styleType": "dark",
                "topLayerGroups": {},
                "visibleLayerGroups": {
                    "label": True,
                    "road": True,
                    "border": True,  # Włącz granice
                    "building": True,
                    "water": True,
                    "land": True
                }
            }
        }
    }

    # Stwórz mapę z określoną wysokością
    map_1 = KeplerGl(height=800)

    # Dodaj dane do mapy
    map_1.add_data(data=gdf, name='population_data')
    map_1.add_data(data=countries, name="Countries")
    map_1.add_data(data=cities, name="Cities")

    # Zastosuj konfigurację
    map_1.config = config

    # Zapisz do pliku HTML
    map_1.save_to_html(file_name=filename)

    print("Mapa została zapisana! Otwórz plik kepler_map_seir.html w przeglądarce.")

testing = False
if testing:
    simulation = PandemicSpreadSimulation()
    create_map_with_seir(simulation.gdf)

In [14]:
simulation = PandemicSpreadSimulation()
create_map_with_seir(simulation.gdf, 'day0.html')
for i in range(2):
    simulation.update_simulation()
    create_map_with_seir(simulation.gdf, f'day{i + 1}.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to day0.html!
Mapa została zapisana! Otwórz plik kepler_map_seir.html w przeglądarce.
Neighbor infections processed.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to day1.html!
Mapa została zapisana! Otwórz plik kepler_map_seir.html w przeglądarce.
Neighbor infections processed.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to day2.html!
Mapa została zapisana! Otwórz plik kepler_map_seir.html w przeglądarce.
